In [1]:
import numpy as np
import pandas as pd
from typing import Dict
import selection_methods as smtd

In [2]:
subclasses = set()
work = [smtd.Feature_Selection]
while work:
    parent = work.pop()
    for child in parent.__subclasses__():
        if child not in subclasses:
            subclasses.add(child)
            work.append(child)
METHODS = list(subclasses)

In [3]:
METHODS

[selection_methods.RFE_Selection,
 selection_methods.Lasso_Selection,
 selection_methods.Xgb_Selection,
 selection_methods.Shap_Selection,
 selection_methods.GBM_Selection,
 selection_methods.Catboost_Selection,
 selection_methods.Rf_Selection,
 selection_methods.MRMR,
 selection_methods.PCA_Selection]

In [4]:
df = pd.concat((pd.read_csv('../data/Telco-Customer-Churn-encoded-data-FE.csv'), pd.read_csv('../data/Telco-Customer-Churn-encoded-label.csv')), axis=1)

In [5]:
df.replace([np.inf, -np.inf], 0, inplace=True)


In [6]:
df.columns

Index(['gender_Male', 'SeniorCitizen_1', 'Partner_Yes', 'Dependents_Yes',
       'PhoneService_Yes', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
  

In [7]:
data = df.drop('Churn', axis=1)
target = df['Churn']

In [8]:
importances = pd.DataFrame()

for _ in METHODS:
    method_name = _.__name__
    print(f'Method: {method_name}')
    model = _(data, target=target)
    model.fit()
    print(f'Feature importances:\n {model.get_importances()}')
    new_df = pd.DataFrame(model.get_importances(), index=[method_name])
    importances = pd.concat([importances, new_df])

Method: RFE_Selection
Feature importances:
 {'TotalCharges': 23, 'TotalCharges/tenure': 22, 'MonthlyCharges': 21, 'DeviceProtection_Yes': 20, 'tenure': 19, 'gender_Male': 18, 'Partner_Yes': 17, 'Dependents_Yes': 16, 'DeviceProtection_No': 15, 'SeniorCitizen_1': 14, 'tenure_group': 13, 'PaperlessBilling_Yes': 12, 'Monthly/Total_Charges': 11, 'InternetService_No': 10, 'OnlineBackup_No': 9, 'OnlineBackup_Yes': 8, 'TechSupport_Yes': 7, 'TechSupport_No': 6, 'StreamingTV_Yes': 5, 'StreamingTV_No': 4, 'MultipleLines_No phone service': 3, 'OnlineSecurity_No internet service': 2, 'PhoneService_Yes': 1, 'MultipleLines_No': 1, 'MultipleLines_Yes': 1, 'InternetService_DSL': 1, 'InternetService_Fiber optic': 1, 'OnlineSecurity_No': 1, 'OnlineSecurity_Yes': 1, 'OnlineBackup_No internet service': 1, 'DeviceProtection_No internet service': 1, 'TechSupport_No internet service': 1, 'StreamingTV_No internet service': 1, 'StreamingMovies_No': 1, 'StreamingMovies_No internet service': 1, 'StreamingMovies_Y

100%|██████████| 43/43 [00:07<00:00,  5.58it/s]

Feature importances:
 {'Contract_Month-to-month': 44867693.57400776, 'PaymentMethod_Mailed check': 1837.5254123490213, 'MonthlyCharges': 1411.6780745684919, 'Monthly/Total_Charges': 957.3017786689212, 'tenure': 949.7403718978271, 'OnlineSecurity_No': 948.9391860971717, 'TechSupport_No': 929.3885761034352, 'InternetService_Fiber optic': 908.9493186361709, 'PaymentMethod_Electronic check': 862.2263658816254, 'tenure_group': 796.4821944497849, 'Contract_Two year': 683.9243337226302, 'OnlineBackup_No': 678.9734214030833, 'SeniorCitizen_1': 549.4572788429726, 'DeviceProtection_No': 528.3549477914346, 'PaperlessBilling_Yes': 527.4132227908558, 'InternetService_No': 477.35689337200455, 'Contract_One year': 442.2081325139821, 'Dependents_Yes': 429.3188144462161, 'OnlineSecurity_No internet service': 414.9617656878607, 'OnlineBackup_No internet service': 394.29273451834837, 'Partner_Yes': 382.0462011256756, 'DeviceProtection_No internet service': 371.67176208815954, 'OnlineSecurity_Yes': 358.62

In [9]:
importances

,TotalCharges,TotalCharges/tenure,MonthlyCharges,DeviceProtection_Yes,tenure,gender_Male,Partner_Yes,Dependents_Yes,DeviceProtection_No,SeniorCitizen_1,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
RFE_Selection,2.300000e+01,2.200000e+01,2.100000e+01,2.000000e+01,1.900000e+01,18.000000,17.000000,16.000000,1.500000e+01,14.000000,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Lasso_Selection,-8.180496e-05,3.949209e-03,3.218145e-03,-0.000000e+00,-2.120989e-03,-0.000000,-0.000000,-0.000000,0.000000e+00,0.000000,...,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00
Xgb_Selection,9.501263e-03,9.169737e-03,9.158291e-03,5.777318e-03,8.393361e-03,0.007556,0.005589,0.008352,5.083259e-03,0.006929,...,1.019804e-02,0.000000e+00,1.200656e-02,3.334700e-01,1.435750e-02,1.638632e-02,6.945182e-03,6.517187e-03,1.149953e-02,6.196208e-03
Shap_Selection,2.975569e-01,3.245621e-01,3.771491e-01,1.593113e-02,2.831978e-01,0.065575,0.040230,0.045734,3.005718e-02,0.097508,...,4.240838e-02,0.000000e+00,6.432658e-02,8.437142e-01,1.293914e-01,2.006349e-01,5.737625e-02,3.466719e-02,1.914869e-01,5.439763e-02
GBM_Selection,4.208893e-02,4.638008e-02,4.124097e-02,4.574179e-04,9.181371e-03,0.001737,0.000000,0.002904,0.000000e+00,0.007842,...,0.000000e+00,4.716326e-05,4.761454e-03,3.792985e-01,5.203307e-03,6.087090e-03,2.211407e-03,6.784975e-04,3.894626e-02,1.083515e-03
Catboost_Selection,1.006677e+01,9.637562e+00,1.039317e+01,5.788174e-01,6.336968e+00,2.490836,1.565176,1.750581,4.854160e-01,2.225095,...,5.655437e-01,5.717536e-01,9.005274e-01,5.485722e+00,1.586317e+00,3.189817e+00,1.519586e+00,1.361056e+00,2.425483e+00,1.413549e+00
Rf_Selection,1.091628e-01,1.087447e-01,1.045853e-01,9.322163e-03,8.300387e-02,0.021131,0.016881,0.014303,1.166995e-02,0.015071,...,8.357130e-03,4.532442e-03,9.038437e-03,4.916054e-02,7.872967e-03,1.748400e-02,9.733994e-03,1.008756e-02,2.201728e-02,9.534494e-03
MRMR,2.998323e+02,2.757412e+02,1.411678e+03,2.829668e+01,9.497404e+02,95.489931,382.046201,429.318814,5.283549e+02,549.457279,...,1.694544e+02,3.191564e+02,5.758458e+01,4.486769e+07,4.422081e+02,6.839243e+02,3.273879e+02,3.380190e+02,8.622264e+02,1.837525e+03
PCA_Selection,9.148514e-33,9.148514e-33,9.148514e-33,2.776109e-08,9.148514e-33,0.999757,0.000018,0.000001,3.917533e-08,0.000222,...,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33


In [10]:
importances.index

Index(['RFE_Selection', 'Lasso_Selection', 'Xgb_Selection', 'Shap_Selection',
       'GBM_Selection', 'Catboost_Selection', 'Rf_Selection', 'MRMR',
       'PCA_Selection'],
      dtype='object')

In [11]:
for ind in importances.index:
    values_list = importances.loc[ind].tolist()
    non_zero_abs_values = [abs(value) for value in values_list if value != 0]
    smallest_non_zero = min(non_zero_abs_values)
    for col in importances.columns:
        if importances.loc[ind, col] == 0.0:
            importances.loc[ind, col] = smallest_non_zero / 100

In [12]:
importances

,TotalCharges,TotalCharges/tenure,MonthlyCharges,DeviceProtection_Yes,tenure,gender_Male,Partner_Yes,Dependents_Yes,DeviceProtection_No,SeniorCitizen_1,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
RFE_Selection,2.300000e+01,2.200000e+01,2.100000e+01,2.000000e+01,1.900000e+01,1.800000e+01,1.700000e+01,1.600000e+01,1.500000e+01,1.400000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Lasso_Selection,-8.180496e-05,3.949209e-03,3.218145e-03,8.180496e-07,-2.120989e-03,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,...,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07,8.180496e-07
Xgb_Selection,9.501263e-03,9.169737e-03,9.158291e-03,5.777318e-03,8.393361e-03,7.555894e-03,5.589140e-03,8.352404e-03,5.083259e-03,6.928868e-03,...,1.019804e-02,3.924814e-05,1.200656e-02,3.334700e-01,1.435750e-02,1.638632e-02,6.945182e-03,6.517187e-03,1.149953e-02,6.196208e-03
Shap_Selection,2.975569e-01,3.245621e-01,3.771491e-01,1.593113e-02,2.831978e-01,6.557450e-02,4.022958e-02,4.573368e-02,3.005718e-02,9.750839e-02,...,4.240838e-02,8.343856e-05,6.432658e-02,8.437142e-01,1.293914e-01,2.006349e-01,5.737625e-02,3.466719e-02,1.914869e-01,5.439763e-02
GBM_Selection,4.208893e-02,4.638008e-02,4.124097e-02,4.574179e-04,9.181371e-03,1.736675e-03,3.848828e-07,2.904435e-03,3.848828e-07,7.841896e-03,...,3.848828e-07,4.716326e-05,4.761454e-03,3.792985e-01,5.203307e-03,6.087090e-03,2.211407e-03,6.784975e-04,3.894626e-02,1.083515e-03
Catboost_Selection,1.006677e+01,9.637562e+00,1.039317e+01,5.788174e-01,6.336968e+00,2.490836e+00,1.565176e+00,1.750581e+00,4.854160e-01,2.225095e+00,...,5.655437e-01,5.717536e-01,9.005274e-01,5.485722e+00,1.586317e+00,3.189817e+00,1.519586e+00,1.361056e+00,2.425483e+00,1.413549e+00
Rf_Selection,1.091628e-01,1.087447e-01,1.045853e-01,9.322163e-03,8.300387e-02,2.113092e-02,1.688149e-02,1.430315e-02,1.166995e-02,1.507133e-02,...,8.357130e-03,4.532442e-03,9.038437e-03,4.916054e-02,7.872967e-03,1.748400e-02,9.733994e-03,1.008756e-02,2.201728e-02,9.534494e-03
MRMR,2.998323e+02,2.757412e+02,1.411678e+03,2.829668e+01,9.497404e+02,9.548993e+01,3.820462e+02,4.293188e+02,5.283549e+02,5.494573e+02,...,1.694544e+02,3.191564e+02,5.758458e+01,4.486769e+07,4.422081e+02,6.839243e+02,3.273879e+02,3.380190e+02,8.622264e+02,1.837525e+03
PCA_Selection,9.148514e-33,9.148514e-33,9.148514e-33,2.776109e-08,9.148514e-33,9.997572e-01,1.847671e-05,1.000797e-06,3.917533e-08,2.222596e-04,...,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33,9.148514e-33


In [13]:
zeros_count = (importances == 0).sum().sum()
zeros_count

0

In [14]:
importances.drop('PCA_Selection', axis='index')

multiplications = importances.apply(lambda x: abs(x).prod(), axis=0)

# Sort the values in descending order
multiplications_sorted = multiplications.sort_values(ascending=False)

# Create a DataFrame from the sorted values
multiplications_df = pd.DataFrame({'Column': multiplications_sorted.index, 'Multiplication': multiplications_sorted.values})

# Save the DataFrame as a CSV file
multiplications_df.to_csv('importance_values.csv', index=False)

# Print the DataFrame
print(multiplications_df)

                                     Column  Multiplication
0                               gender_Male    6.366593e-11
1                           SeniorCitizen_1    2.485007e-13
2               InternetService_Fiber optic    9.878054e-15
3                         OnlineSecurity_No    9.971156e-16
4                            TechSupport_No    8.432137e-16
5                            Dependents_Yes    1.562243e-16
6                           OnlineBackup_No    1.379537e-17
7                          MultipleLines_No    5.023242e-19
8                               Partner_Yes    2.244718e-19
9                          OnlineBackup_Yes    4.049447e-20
10                           StreamingTV_No    7.521617e-21
11                          TechSupport_Yes    4.945972e-21
12                     DeviceProtection_Yes    2.919617e-21
13                         PhoneService_Yes    1.991751e-21
14                        MultipleLines_Yes    1.657404e-21
15                       InternetService